---
# **RUN on video**
---

In [1]:
%load_ext autoreload
%autoreload 2

import pyVHR as vhr
import numpy as np
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.plot.visualize import *
import os
import plotly.express as px
from pyVHR.utils.errors import getErrors, printErrors, displayErrors

import constants
import pandas as pd

vhr.plot.VisualizeParams.renderer = 'vscode' 

# LGI-PPGI Baseline

TODO why is this weird now, only 40s although seconds = 0 total_frames ?
is it because im not printing error ? 

In [2]:
# -- SET DATASET

dataset_name = 'LGI_PPGI'                   # the name of the python class handling it 
video_DIR, BVP_DIR = constants.get_dataset_paths(dataset_name)

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

0 D:/datasets_rppg/LGI_PPGI\alex\alex_gym\cv_camera_sensor_stream_handler.avi
1 D:/datasets_rppg/LGI_PPGI\alex\alex_resting\cv_camera_sensor_stream_handler.avi
2 D:/datasets_rppg/LGI_PPGI\alex\alex_rotation\cv_camera_sensor_stream_handler.avi
3 D:/datasets_rppg/LGI_PPGI\alex\alex_talk\cv_camera_sensor_stream_handler.avi
4 D:/datasets_rppg/LGI_PPGI\angelo\angelo_gym\cv_camera_sensor_stream_handler.avi
5 D:/datasets_rppg/LGI_PPGI\angelo\angelo_resting\cv_camera_sensor_stream_handler.avi
6 D:/datasets_rppg/LGI_PPGI\angelo\angelo_rotation\cv_camera_sensor_stream_handler.avi
7 D:/datasets_rppg/LGI_PPGI\angelo\angelo_talk\cv_camera_sensor_stream_handler.avi
8 D:/datasets_rppg/LGI_PPGI\cpi\cpi_gym\cv_camera_sensor_stream_handler.avi
9 D:/datasets_rppg/LGI_PPGI\cpi\cpi_resting\cv_camera_sensor_stream_handler.avi
10 D:/datasets_rppg/LGI_PPGI\cpi\cpi_rotation\cv_camera_sensor_stream_handler.avi
11 D:/datasets_rppg/LGI_PPGI\cpi\cpi_talk\cv_camera_sensor_stream_handler.avi
12 D:/datasets_rppg/LGI_

In [3]:
# -- PARAMETER SETTING and GT

wsize = 8        # seconds of video processed (with overlapping) for each estimate 
video_idx = 13    # index of the video to be processed
fname = dataset.getSigFilename(video_idx)
sigGT = dataset.readSigfile(fname)
test_bvp = sigGT.data
bpmGT, timesGT = sigGT.getBPM(wsize)  # GT
videoFileName = dataset.getVideoFilename(video_idx)
print('Video to process: ', videoFileName)
fps = vhr.extraction.get_fps(videoFileName)
print('Video frame rate: ',fps)

Video to process:  D:/datasets_rppg/LGI_PPGI\david\david_resting\cv_camera_sensor_stream_handler.avi
Video frame rate:  25.0


In [6]:
def save_results(RMSE, MAE, MAX, PCC, CCC, SNR, roi_approach, bpm_est, videoFileName):
    return {'RMSE':RMSE[0], 'MAE':MAE[0], 'MAX':MAX[0], 'PCC':PCC[0], 'CCC':CCC[0], 'SNR':SNR[0],'ROI':roi_approach+'_'+bpm_est,'Video':'-'.join(videoFileName.split('\\')[-1].split('-')[1:])[:-4]}


def run_pipeline(roi_approach, bpm_est):

    bvps, timesES, bpmES = pipe.run_on_video(
        videoFileName,
        winsize=wsize, 
        roi_method='convexhull',
        roi_approach=roi_approach,
        method='cpu_CHROM',
        estimate=bpm_est,
        patch_size=30, 
        total_frames = seconds*fps,
        RGB_LOW_HIGH_TH=(5,230),
        Skin_LOW_HIGH_TH=(5,230),
        pre_filt=True,
        post_filt=True,
        cuda=True, 
        verb=False
    )
    
    return bvps, timesES, bpmES

# -- PARAMETER SETTING and GT
fname = dataset.getSigFilename(0)
sigGT = dataset.readSigfile(fname)
test_bvp = sigGT.data
bpmGT, timesGT = sigGT.getBPM(wsize)  # GT
videoFileName = dataset.getVideoFilename(video_idx)
print('Video to process: ', videoFileName)
fps = vhr.extraction.get_fps(videoFileName)
seconds = 0 #0: all video

bpmES_list, timesES_list, bpmGT_list, timesGT_list = [], [], [], []

for roi_approach in ['holistic', 'patches']:
    pipe = Pipeline()          # object to execute the pipeline
    if roi_approach == 'patches':
        for bpm_est in ['median', 'clustering']:
            bvps, timesES, bpmES = run_pipeline(roi_approach, bpm_est)
    else:
        bpm_est = 'median'
        bvps, timesES, bpmES = run_pipeline(roi_approach, bpm_est)
    RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(bvps, fps, bpmES, bpmGT, timesES, timesGT)
    print(f"Video {video_idx}: ROI approach: {roi_approach}, BPM estimation: {bpm_est}, RMSE: {RMSE}, MAE: {MAE},PCC: {PCC},SNR: {SNR}")
    timesES_list.append(timesES)
    bpmES_list.append(bpmES)
    bpmGT_list.append(bpmGT)
    timesGT_list.append(timesGT)

# -- PLOT RESULTS # 0: holistic 1: patches median 2: patches clustering
displayErrors(bpmES_list[0], bpmGT_list[0], timesES_list[0], timesGT_list[0])

Video to process:  D:/datasets_rppg/LGI_PPGI\david\david_resting\cv_camera_sensor_stream_handler.avi
0
# CUDA devices:  2
# device number  0 :  NVIDIA GeForce RTX 3080
# device number  1 :  NVIDIA GeForce RTX 3080
Video 13: ROI approach: holistic, BPM estimation: median, RMSE: [30.44382343], MAE: [30.06099232],PCC: [0.04425209],SNR: [-14.45596177]
0
# CUDA devices:  2
# device number  0 :  NVIDIA GeForce RTX 3080
# device number  1 :  NVIDIA GeForce RTX 3080
0
# CUDA devices:  2
# device number  0 :  NVIDIA GeForce RTX 3080
# device number  1 :  NVIDIA GeForce RTX 3080
Video 13: ROI approach: patches, BPM estimation: clustering, RMSE: [30.38786964], MAE: [30.0095943],PCC: [0.04266142],SNR: [-7.90300197]


# Compare EVM 

In [8]:
# -- SET DATASET

name = 'david'
dataset_name = 'LGI_PPGI_EVM'                   # the name of the python class handling it 
video_DIR = f'D:/EVM/LGI_PPGI_EVM/EVM_{name}' # f'D:/EVM/LGI_PPGI_EVM/'
BVP_DIR =  f'D:/EVM/LGI_PPGI_EVM/EVM_{name}'

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

0 D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-25-level-2-chromAtn-0.5.avi
1 D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-25-level-2-chromAtn-0.avi
2 D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-25-level-2-chromAtn-1.avi
3 D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-25-level-4-chromAtn-0.5.avi
4 D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-25-level-4-chromAtn-0.avi
5 D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-25-level-4-chromAtn-1.avi
6 D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-25-level-6-chromAtn-0.5.avi
7 D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-25-level-6-chromAtn-0.avi
8 D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-25-level-6-chromAtn-1.avi
9 D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-50-level-2-chromAtn-0.5.avi
10 D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-50-level-2-chromAtn-0.avi
11 D:/EVM/LGI_PPGI_EVM/

In [16]:
wsize = 8        # seconds of video processed (with overlapping) for each estimate 
method = 'cpu_CHROM'       # one of the methods implemented in pyVHR
seconds = 0 # 0: entire video
results = []

def save_results(RMSE, MAE, MAX, PCC, CCC, SNR, roi_approach, bpm_est, videoFileName):
    return {'RMSE':RMSE[0], 'MAE':MAE[0], 'MAX':MAX[0], 'PCC':PCC[0], 'CCC':CCC[0], 'SNR':SNR[0],'ROI':roi_approach+'_'+bpm_est,'Video':'-'.join(videoFileName.split('\\')[-1].split('-')[1:])[:-4]}


def run_pipeline(roi_approach, bpm_est, video_idx, videoFileName, results):
    try:
        bvps, timesES, bpmES = pipe.run_on_video(
            videoFileName,
            winsize=wsize, 
            roi_method='convexhull',
            roi_approach=roi_approach,
            method='cpu_CHROM',
            estimate=bpm_est,
            patch_size=40, 
            total_frames = seconds*fps,
            RGB_LOW_HIGH_TH=(5,230),
            Skin_LOW_HIGH_TH=(5,230),
            pre_filt=True,
            post_filt=True,
            cuda=True, 
            verb=False
        )

        # ERRORS
        RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(bvps, fps, bpmES, bpmGT, timesES, timesGT)
        print(f"Video {video_idx}: ROI approach: {roi_approach}, BPM estimation: {bpm_est}, RMSE: {RMSE}, MAE: {MAE},PCC: {PCC},SNR: {SNR}")
        results.append(save_results(RMSE, MAE, MAX, PCC, CCC, SNR, roi_approach, bpm_est, videoFileName))
    except:
        print(f"Video {video_idx} {videoFileName}: ROI approach: {roi_approach}, BPM estimation: {bpm_est}, ERROR")
        results.append({'RMSE':np.nan, 'MAE':np.nan, 'MAX':np.nan, 'PCC':np.nan, 'CCC':np.nan, 'SNR':np.nan,'ROI':roi_approach+'_'+bpm_est,'Video':'-'.join(videoFileName.split('\\')[-1].split('-')[1:])[:-4]})
    return results 

for video_idx,video in enumerate(allvideo):
    # -- PARAMETER SETTING and GT
    fname = dataset.getSigFilename(0)
    sigGT = dataset.readSigfile(fname)
    test_bvp = sigGT.data
    bpmGT, timesGT = sigGT.getBPM(wsize)  # GT
    videoFileName = dataset.getVideoFilename(video_idx)
    print('Video to process: ', videoFileName)
    fps = vhr.extraction.get_fps(videoFileName)

    for roi_approach in ['holistic', 'patches']:
        pipe = Pipeline()          # object to execute the pipeline
        if roi_approach == 'patches':
            for bpm_est in ['median', 'clustering']:
                results = run_pipeline(roi_approach, bpm_est,video_idx, videoFileName, results)
        else:
            results = run_pipeline(roi_approach, 'median',video_idx, videoFileName, results)


Video to process:  D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-25-level-2-chromAtn-0.5.avi
# CUDA devices:  2
# device number  0 :  NVIDIA GeForce RTX 3080
# device number  1 :  NVIDIA GeForce RTX 3080
Video 0: ROI approach: holistic, BPM estimation: median, RMSE: [1.37980401], MAE: [1.12766335],PCC: [0.8873611],SNR: [9.68501631]
# CUDA devices:  2
# device number  0 :  NVIDIA GeForce RTX 3080
# device number  1 :  NVIDIA GeForce RTX 3080
Video 0: ROI approach: patches, BPM estimation: median, RMSE: [1.42699765], MAE: [1.21404474],PCC: [0.89460664],SNR: [2.67958069]
# CUDA devices:  2
# device number  0 :  NVIDIA GeForce RTX 3080
# device number  1 :  NVIDIA GeForce RTX 3080
Video 0: ROI approach: patches, BPM estimation: clustering, RMSE: [1.44718136], MAE: [1.20294744],PCC: [0.89546061],SNR: [2.67958069]
Video to process:  D:/EVM/LGI_PPGI_EVM/EVM_david\evm_david-from-0.5-to-4-alpha-25-level-2-chromAtn-0.avi
# CUDA devices:  2
# device number  0 :  NVIDIA GeForce RTX 3

# Save Results

In [17]:
import pandas as pd
df = pd.DataFrame(results)

In [18]:
df

,RMSE,MAE,MAX,PCC,CCC,SNR,ROI,Video
0,1.379804,1.127663,3.171875,0.887361,0.800732,9.685016,holistic_median,from-0.5-to-4-alpha-25-level-2-chromAtn-0.5
1,1.426998,1.214045,3.171875,0.894607,0.789739,2.679581,patches_median,from-0.5-to-4-alpha-25-level-2-chromAtn-0.5
2,1.447181,1.202947,3.171875,0.895461,0.783872,2.679581,patches_clustering,from-0.5-to-4-alpha-25-level-2-chromAtn-0.5
3,18.633852,10.225616,72.554688,-0.307630,-0.069713,1.052671,holistic_median,from-0.5-to-4-alpha-25-level-2-chromAtn-0
4,2.626669,1.984345,7.384766,0.455585,0.432687,-0.153568,patches_median,from-0.5-to-4-alpha-25-level-2-chromAtn-0
...,...,...,...,...,...,...,...,...
76,1.943725,1.693004,3.904297,0.868469,0.681605,-0.765189,patches_median,from-0.5-to-4-alpha-75-level-6-chromAtn-0
77,1.489458,1.277403,3.171875,0.838750,0.771593,-0.765189,patches_clustering,from-0.5-to-4-alpha-75-level-6-chromAtn-0
78,1.506474,1.247337,3.171875,0.865546,0.766535,8.399723,holistic_median,from-0.5-to-4-alpha-75-level-6-chromAtn-1
79,1.519150,1.259973,3.171875,0.863113,0.762105,6.064329,patches_median,from-0.5-to-4-alpha-75-level-6-chromAtn-1


In [21]:
name = videoFileName.split('\\')[-1].split('-')[0].split('_')[-1]
file_res = f'../datasets/{name}_evm.csv'
print(file_res)
df.to_csv(file_res, index=False)
# df.to_csv(file_res, mode='a', header=False, index=False)

../datasets/david_evm.csv


In [16]:
name = videoFileName.split('\\')[-1].split('-')[0].split('_')[-1]
file_res = f'../datasets/{name}_evm.csv'
x = pd.read_csv(file_res)  

In [18]:
print("Name: ", name)
x

Name:  david


,RMSE,MAE,MAX,PCC,CCC,SNR,ROI,Video
0,1.479878,1.223444,3.171875,0.857412,0.746696,9.802536,holistic_median,from-0.5-to-4-alpha-50-level-4-chromAtn-1
1,1.469627,1.221508,3.171875,0.865463,0.745933,3.540380,patches_median,from-0.5-to-4-alpha-50-level-4-chromAtn-1
2,1.425396,1.173725,3.171875,0.882572,0.766362,3.540380,patches_clustering,from-0.5-to-4-alpha-50-level-4-chromAtn-1
